# 02. langchain的快速入门
https://python.langchain.com/docs/get_started/quickstart

In [3]:
from langchain_openai import OpenAI

# llm = OpenAI(openai_api_key="...") # 如果你有openai的api key，可以在这里填入
llm = OpenAI() # 这种写法是通过系统环境变量获取openai的api key

#### 环境变量设置
第一步，按下“win+r”键，打开运行框，

第二步，输入命令“control system”,

第三步，在控制面板主页，打开“高级系统设置”，

第四步，在高级系统设置界面，选择“环境变量”选项，

第五步，根据需求进行设置。

In [10]:
#设置代理
import os

os.environ['http_proxy'] = 'http://127.0.0.1:7809'
os.environ['https_proxy'] = 'http://127.0.0.1:7809'


## 2.1. 构建应用程序
现在我们可以开始构建语言模型应用程序了。LangChain提供了许多可用于构建语言模型应用程序的模块。模块可以在简单的应用程序中作为独立模块使用，并且可以组合用于更复杂的用例。在本节中，我们将介绍如何使用LangChain的模块来构建一个简单的应用程序。

LangChain应用程序的核心构建块是LLMChain。这结合了三件事：

    1.LLM：语言模型是这里的核心推理引擎。为了使用 LangChain，您需要了解不同类型的语言模型以及如何使用它们。

    2.提示模板：这为语言模型提供说明。这控制语言模型输出的内容，因此了解如何构造提示和不同的提示策略至关重要。

    3.输出解析器：这些将来自LLM的原始响应转换为更可行的格式，从而可以轻松使用下游的输出。

我们将单独介绍这三个组件，然后介绍组合所有这些组件的LLMChain。

了解这些概念将为您能够使用和自定义 LangChain 应用程序做好准备。

大多数LangChain应用程序都允许您配置LLM和/或使用的提示，因此知道如何利用这一点将是一个很大的推动因素。

### 2.1.1. LLMs
有两种类型的语言模型，在LangChain中称为：
```
llms：这是一个语言模型，它将字符串作为输入并返回一个字符串
chat_models：这是一种语言模型，它将消息列表作为输入并返回消息
```

LLM 的输入/输出简单易懂 - 一个字符串。

但是chat_models呢？

那里的输入是ChatMessage的列表 ，输出是单个ChatMessage .

一个ChatMessage有两个必需的组件：
```
content ：这是消息的内容。
role ：这是来自的 ChatMessage 实体的角色。
```
LangChain 提供了几个对象来轻松区分不同的角色：
```
HumanMessage ： ChatMessage来自人类/用户。

AIMessage ： ChatMessage来自AI/助手。

SystemMessage ：来自系统的ChatMessage 。

FunctionMessage ：来自函数调用的ChatMessage 。
```

LangChain 为两者公开了一个标准接口，但了解这种差异以便为给定语言模型构造提示很有用。LangChain 公开的标准接口有两种方法：

invoke ：接受字符串，返回字符串

invoke ：接收消息列表，返回消息。

In [6]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI


llm = OpenAI()
chat_model = ChatOpenAI()
llm.invoke("你好!")
# chat_model.invoke("hi!")

'\n你好！我是一个人工智能助手，很高兴认识你。有什么可以帮到你的吗？'

In [7]:
chat_model.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9d51be5e-407a-4e03-bc90-41b47927bea8-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [22]:
llm.invoke("hi!")

'\n\nHola!\n\n¿Cómo estás?\n\nEstoy bien, gracias. ¿Y tú?'

In [8]:
text = "What would be a good company name for a company that makes colorful socks?请用中文回答"

llm.invoke(text)
# >> Feetful of Fun

'\n\n七彩袜子有限公司'

In [9]:
chat_model.invoke(text)
# >> Socks O'Color

AIMessage(content='缤纷袜业', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 29, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b3c23c62-f88d-45fd-bb59-7c00afd0dc5e-0', usage_metadata={'input_tokens': 29, 'output_tokens': 7, 'total_tokens': 36})

In [15]:
from langchain.schema import HumanMessage

# text = "What would be a good company name for a company that makes colorful socks?"
text = "怎样才能年入百万？"

messages = [HumanMessage(content=text)]

In [9]:
chat_model.invoke(messages)
# >> Socks O'Color

AIMessage(content='Rainbow Threads', additional_kwargs={}, example=False)

In [10]:
llm.invoke(messages)
# >> Feetful of Fun

AIMessage(content='\n\nSocktacular!', additional_kwargs={}, example=False)

对于这两种方法，还可以将参数作为关键字参数传入。

例如，您可以传入 temperature=0 以根据对象的配置调整使用的温度。在运行时传入的任何值都将始终覆盖对象配置的内容。

In [18]:
chat_model.invoke(messages,temperature=0)

AIMessage(content='要实现年入百万的目标，需要具备以下几个关键要素：\n\n1. 找到一个高收入的职业或创业机会：选择一个高薪职业或创业项目，比如医生、律师、投资者、企业家等，能够带来稳定且丰厚的收入。\n\n2. 不断提升自己的技能和知识：通过学习、培训和实践，不断提升自己的专业技能和知识水平，以提高自己在职场上的竞争力和价值。\n\n3. 善于投资和理财：学会理财规划和投资，让自己的资产增值，实现财务自由和财富积累。\n\n4. 勤奋努力和坚持不懈：要有足够的毅力和耐心，坚持不懈地努力工作和追求目标，不断挑战自己，才能实现年入百万的目标。\n\n5. 寻找多元化的收入来源：除了主要的工作或创业项目外，还可以寻找其他的收入来源，比如投资、副业、兼职等，以增加收入来源和多样化风险。\n\n总之，要实现年入百万的目标，需要具备正确的职业选择、不断提升自己的能力、善于理财和投资、坚持努力和寻找多元化的收入来源等要素。只有全方位地提升自己，才能实现财务自由和财富积累。', response_metadata={'token_usage': {'completion_tokens': 471, 'prompt_tokens': 19, 'total_tokens': 490}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cdea10cc-38cf-4f1e-8be1-9b77d51125b1-0', usage_metadata={'input_tokens': 19, 'output_tokens': 471, 'total_tokens': 490})

In [17]:
chat_model.invoke(messages,temperature=0.5)

AIMessage(content='要年入百万，通常需要具备以下几个条件：\n\n1. 选择一个高收入的行业或职业：选择一个高薪行业或职业，比如金融、科技、医疗等，能够为你提供更多赚钱的机会。\n\n2. 不断提升自己的技能和知识：不断学习和提升自己的技能和知识，可以让你在职场上更有竞争力，从而获得更高的薪水。\n\n3. 创业或投资：创业或投资是实现年入百万的有效途径之一。通过创业或投资，可以获得更高的回报和收入。\n\n4. 节约开支和理性消费：要年入百万，不仅需要增加收入，还需要控制开支，节约开支和理性消费是实现财务自由的重要前提。\n\n5. 善于理财和投资：善于理财和投资可以让你的资产增值，获得更多的财富积累。\n\n总的来说，要年入百万，需要不断提升自己的能力和技能，选择合适的行业和职业，控制开支，理性消费，创业或投资等多种因素综合作用。', response_metadata={'token_usage': {'completion_tokens': 377, 'prompt_tokens': 19, 'total_tokens': 396}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0fc4eaf3-de81-4346-9a38-684c9661b7fb-0', usage_metadata={'input_tokens': 19, 'output_tokens': 377, 'total_tokens': 396})

In [16]:
chat_model.invoke(messages,temperature=1)

AIMessage(content='要想年入百万，通常需要具备以下条件和做法：\n\n1. 拥有高收入职业：选择一份高薪职业或者行业，比如金融、科技、医疗等领域的高薪工作。\n\n2. 创业或投资：创业或投资是实现年入百万的一种途径，可以寻找有潜力的项目或领域进行投资或自主创业。\n\n3. 学习和提升自我：不断学习和提升自己的能力，增强自己在职场上的竞争力，从而获得更高的报酬。\n\n4. 精打细算：在日常生活中控制开支，合理规划资金运用，避免浪费，增加储蓄和投资的机会。\n\n5. 多元化收入来源：除了主要工作外，还可以通过副业、兼职、投资等方式获取额外收入，实现收入的多元化。\n\n6. 善于理财：学会理财和投资管理，让资金更有效地运转和增值，实现年入百万的目标。\n\n总之，要年入百万需要有正确的职业选择、积极进取的心态、精明的理财规划和持续的努力和学习。', response_metadata={'token_usage': {'completion_tokens': 391, 'prompt_tokens': 19, 'total_tokens': 410}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-573be3a4-5882-4cc8-ac82-54a2c3f91245-0', usage_metadata={'input_tokens': 19, 'output_tokens': 391, 'total_tokens': 410})

### 2.1.2. 提示模板
大多数LLM应用程序不会将用户输入直接传递到LLM。
通常，他们会将用户输入添加到称为提示模板的较大文本中，该文本为手头的特定任务提供额外的上下文。

In [14]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

提示模板还可用于生成消息列表。在这种情况下，提示不仅包含有关内容的信息，还包含每条消息（其角色，其在列表中的位置等）在这里，最常发生的是聊天提示模板是聊天消息模板的列表。
每个聊天消息模板都包含有关如何格式化该聊天消息的说明 - 它的角色，然后是它的内容。

In [16]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt = chat_prompt.format_messages(input_language="English", output_language="chinese", text="I love programming.")

chat_prompt

[SystemMessage(content='You are a helpful assistant that translates English to chinese.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

In [17]:
llm.invoke(chat_prompt)

AIMessage(content='\n\n系統：我愛編程。', additional_kwargs={}, example=False)

### 2.1.3. 输出解析器
输出解析器将LLM的原始输出转换为可以在下游使用的格式。输出分析器的主要类型很少，包括：


从LLM转换文本->结构化信息（例如JSON）

将聊天消息转换为字符串

将调用返回的额外信息转换为字符串，而不是消息（如 OpenAI 函数调用）。

In [18]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")


['hi', 'bye']

### 2.1.4. 大语言模型链
我们现在可以将所有这些组合成一个链。此链将获取输入变量，将这些变量传递给提示模板以创建提示，将提示传递给 LLM，然后通过（可选）输出分析器传递输出。这是捆绑模块化逻辑的便捷方法。让我们看看它的实际效果！

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("colors")
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['red', 'blue', 'green', 'yellow', 'orange']